**PDF 잘라와서 sample.pdf에 넣고 embedding model(KoSimCSE-roberta)통해**

**vector화 한 후 retrieving 하는 것 까지 테스트 해 볼거임**

In [1]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document


In [2]:
sentences = ['치타가 들판을 가로 질러 먹이를 쫓는다.',
             '치타 한 마리가 먹이 뒤에서 달리고 있다.',
             '원숭이 한 마리가 드럼을 연주한다.']

documents = []

for sentence in sentences :
    documents.append(Document(page_content=sentence))

#document type 저장한 list 생성.
print(documents)
print(type(documents))

[Document(page_content='치타가 들판을 가로 질러 먹이를 쫓는다.'), Document(page_content='치타 한 마리가 먹이 뒤에서 달리고 있다.'), Document(page_content='원숭이 한 마리가 드럼을 연주한다.')]
<class 'list'>


In [3]:
import torch

def device_check() : 
    ''' for check cuda availability '''
    device = (
        "cuda"
        if torch.cuda.is_available()
        else "mps"
        if torch.backends.mps.is_available()
        else "cpu"
    )
    return device

In [4]:
embedding = SentenceTransformerEmbeddings(model_name='BM-K/KoSimCSE-roberta', model_kwargs={'device':device_check()}, encode_kwargs={'normalize_embeddings':True})

/Users/choejaehun/anaconda3/envs/ssis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name /Users/choejaehun/.cache/torch/sentence_transformers/BM-K_KoSimCSE-roberta. Creating a new one with MEAN pooling.


In [5]:
embedding #embedding 생성 완료.

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
), model_name='BM-K/KoSimCSE-roberta', cache_folder=None, model_kwargs={'device': 'mps'}, encode_kwargs={'normalize_embeddings': True}, multi_process=False)

In [6]:
vectordb = Chroma.from_documents(documents=documents, embedding=embedding)
chroma_retriever = vectordb.as_retriever(search_type='mmr', search_kwargs={"k": 2}) #retriever 생성

In [7]:
chroma_retriever.invoke("치타는 드럼을 치는가?")

Number of requested results 20 is greater than number of elements in index 3, updating n_results = 3


[Document(page_content='치타 한 마리가 먹이 뒤에서 달리고 있다.'),
 Document(page_content='원숭이 한 마리가 드럼을 연주한다.')]

***결론***

Embedding Function만 쓰면 잘 가져온다. 근데 relevance 기반이라 키워드 이런거는 어떻게 해야 할지 고민해야 할듯.

키워드 search 같은 걸 해야하나?